# SQL Case Study

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite:///data/country_club.db')

In [3]:
def run_query(query_string):
    con = engine.connect()
    result = con.execute(query_string)
    df = pd.DataFrame(result.fetchall())
    df.columns = result.keys()
    con.close()
    return df

In [4]:
query_string_test = """
SELECT *
FROM Bookings;
"""

In [5]:
df_test = run_query(query_string_test)

In [6]:
df_test.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


**Q10:** Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [7]:
query_string_10 = """
SELECT
  name,
  total_revenue
FROM (
  SELECT 
    f.name, 
    SUM(CASE WHEN b.memid = 0 THEN guestcost * slots
          ELSE membercost * slots END ) AS total_revenue
  FROM Bookings AS b
  INNER JOIN Facilities AS f ON b.facid = f.facid
  INNER JOIN Members AS m ON b.memid = m.memid
  GROUP BY name
) AS t
WHERE total_revenue < 1000
ORDER BY total_revenue;
"""

In [8]:
df_q10 = run_query(query_string_10)

In [9]:
df_q10

,name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


**Q11:** Produce a report of members and who recommended them in alphabetic surname,firstname order.

In [10]:
query_string_11 = """
SELECT
  m.surname,
  m.firstname,
  r.surname AS recommender_surname,
  r.firstname AS recommender_firstname
FROM Members as r
INNER JOIN Members as m
ON m.recommendedby = r.memid
ORDER BY m.surname, m.firstname
;
"""

In [11]:
df_q11 = run_query(query_string_11)

In [12]:
df_q11

,surname,firstname,recommender_surname,recommender_firstname
0,Bader,Florence,Stibbons,Ponder
1,Baker,Anne,Stibbons,Ponder
2,Baker,Timothy,Farrell,Jemima
3,Boothe,Tim,Rownam,Tim
4,Butters,Gerald,Smith,Darren
5,Coplin,Joan,Baker,Timothy
6,Crumpet,Erica,Smith,Tracy
7,Dare,Nancy,Joplette,Janice
8,Genting,Matthew,Butters,Gerald
9,Hunt,John,Purview,Millicent


**Q12:** Find the facilities with their usage by member, but not guests.

In [13]:
query_string_12 = """
SELECT 
    f.name, 
    SUM(CASE WHEN b.memid = 0 THEN 0
          ELSE slots END ) AS member_usage
FROM Bookings AS b
INNER JOIN Facilities AS f ON b.facid = f.facid
INNER JOIN Members AS m ON b.memid = m.memid
GROUP BY name;
"""

In [14]:
df_q12 = run_query(query_string_12)

In [15]:
df_q12

,name,member_usage
0,Badminton Court,1086
1,Massage Room 1,884
2,Massage Room 2,54
3,Pool Table,856
4,Snooker Table,860
5,Squash Court,418
6,Table Tennis,794
7,Tennis Court 1,957
8,Tennis Court 2,882


**Q13:** Find the facilities usage by month, but not guests

In [16]:
query_string_13 = """
SELECT 
    f.name,
    SUBSTR(starttime, 6, 2) AS month,
    SUM(CASE WHEN b.memid = 0 THEN 0
          ELSE slots END ) AS member_usage
FROM Bookings AS b
INNER JOIN Facilities AS f ON b.facid = f.facid
INNER JOIN Members AS m ON b.memid = m.memid
GROUP BY name, month
ORDER BY name, month;
"""

In [17]:
df_q13 = run_query(query_string_13)

In [18]:
df_q13

,name,month,member_usage
0,Badminton Court,07,165
1,Badminton Court,08,414
2,Badminton Court,09,507
3,Massage Room 1,07,166
4,Massage Room 1,08,316
5,Massage Room 1,09,402
6,Massage Room 2,07,8
7,Massage Room 2,08,18
8,Massage Room 2,09,28
9,Pool Table,07,110
